In [18]:
import pandas as pd
import vectorbt as vbt
from sqlalchemy.engine import Engine
from sqlalchemy import text
from manage_engine import engine_stockDB

class Connect(vbt.Data):
    """
    DataSource cho vectorbt lấy OHLCV từ bảng eod.eod_stock_v2.

    Yêu cầu:
    - Truyền vào `engine` (SQLAlchemy Engine)
    - Trả về DataFrame index = DateTime, columns = [Open, High, Low, Close, Volume]
    """

    @classmethod
    def download_symbol(
        cls,
        symbol: str,
        *,
        engine: Engine,
        start: str | None = None,
        end: str | None = None,
        **kwargs,
    ) -> pd.DataFrame:
        """
        Hàm này được vectorbt gọi ngầm khi dùng Connect.fetch(...)

        Parameters
        ----------
        symbol : mã cổ phiếu, ví dụ 'HPG'
        engine : SQLAlchemy Engine kết nối tới Postgres
        start, end : chuỗi ngày 'YYYY-MM-DD' hoặc None
        """

        if engine is None:
            raise ValueError("Bạn phải truyền `engine=` (SQLAlchemy Engine) vào Connect.fetch(...)")

        # Base query
        sql = """
        SELECT
            "DateTime" AS dt,
            "Open"     AS "open",
            "High"     AS "high",
            "Low"      AS "low",
            "Close"    AS "close",
            "Volume"   AS "volume"
        FROM eod.eod_stock_v2
        WHERE ticker = :symbol
        """

        params: dict = {"symbol": symbol}

        # Thêm điều kiện ngày nếu có
        if start is not None:
            sql += ' AND "DateTime"::date >= :start'
            params["start"] = start
        if end is not None:
            sql += ' AND "DateTime"::date <= :end'
            params["end"] = end

        sql += ' ORDER BY "DateTime"'

        df = pd.read_sql_query(text(sql), engine, params=params)

        if df.empty:
            # Cho vectorbt biết là symbol này không có dữ liệu
            return df

        # Set index đúng kiểu time series
        df.set_index("dt", inplace=True)
        df.index.name = None  # cho đẹp, không bắt buộc

        # Đảm bảo đúng format OHLCV mà vectorbt thích
        # Cột đã rename sẵn ở query, nên chỉ cần return df
        return df


In [19]:


data = Connect.download(
    ['HPG'],
    engine=engine_stockDB,
    start="2022-01-01",
    end="2024-12-31",
)

df_ohlcv = data.get()



In [2]:
import vectorbtpro as vbt 


In [3]:
client = vbt.TVData.resolve_client(username="hoangmanhhungf", password="Manhhung022193@")

ValueError: {'error': 'Please confirm that you are not a robot by clicking the captcha box.', 'code': 'recaptcha_required'}

In [4]:
data = vbt.YFData.pull("AAPL")

ImportError: Please install yfinance>=0.2.56. See https://pypi.org/project/yfinance/.